In [1]:
# import matplotlib.pyplot as plt
import networkx as nx
import os
import pandas as pd
import pickle

In [2]:
data_path = os.getcwd() + '/Pickle/all_rels.pkl'

In [3]:
df = pd.read_pickle(data_path)

In [4]:
df.head()

,category_id,description2,id,description,entity1_id,is_current,updated_at,entity2_id,currency,start_date,amount,goods,description1,end_date,filings
1,1,Director,1,Allen I Questrom had a position (Director) at...,1006,False,2011-03-13T20:15:59Z,1,None,2007-00-00,NaN,None,Director,2010-06-00,NaN
2,1,Director,2,Linda S Wolf has a position (Director) at Wa...,1007,True,2008-11-05T17:05:10Z,1,None,2005-00-00,NaN,None,Director,None,NaN
3,1,Director,3,Aida M Alvarez had a position (Director) at ...,1008,False,2018-04-23T14:25:16Z,1,None,None,NaN,None,Director,2015-00-00,NaN
4,1,Chairman,4,Gregory Boyd Penner has/had a position (Vice ...,1009,None,2018-04-23T14:54:35Z,1,None,None,NaN,None,Vice Chairman,None,NaN
6,1,Director,6,Christopher J Williams had a position (Direct...,1011,False,2015-03-31T14:56:50Z,1,None,2004-00-00,NaN,None,Director,None,NaN


In [5]:
df.columns

Index(['category_id', 'description2', 'id', 'description', 'entity1_id',
       'is_current', 'updated_at', 'entity2_id', 'currency', 'start_date',
       'amount', 'goods', 'description1', 'end_date', 'filings'],
      dtype='object')

category_id possible values:
1. Position
2. Education
3. Membership
4. Family
5. Donation
6. Transaction
7. Lobbying
8. Social
9. Professional
10. Ownership
11. Hierarchy
12. Generic

In [6]:
df['category_id'].value_counts()

5     835757
1     245885
2      28760
3      27696
4      22266
6      21324
12     15556
11     11828
10     11775
7       6162
9       2244
8       1562
Name: category_id, dtype: int64

In [7]:
def createGraphSingular(data, cat_id):
    subset_df = data[data['category_id'] == cat_id]
    g = nx.from_pandas_edgelist(subset_df, 'entity1_id', 'entity2_id', ['category_id', 'description', 'updated_at'])
    return g

In [8]:
G = createGraphSingular(df, 8)

In [9]:
G[28664]

AtlasView({33448: {'category_id': 8, 'description': 'Eric H Holder Jr  and  Ann Walker Marchant  have/had a social relationship', 'updated_at': '2008-12-06T16:30:54Z'}, 37784: {'category_id': 8, 'description': 'James M Cole  and  Eric H Holder Jr  are longtime friends', 'updated_at': '2009-03-27T08:57:29Z'}})

In [10]:
path = nx.all_simple_paths(G, source = 28664, target = 33448)
print(list(path))

[[28664, 33448]]
